In [1]:
import elasticsearch
import regex
import spacy
from spacy.tokenizer import Tokenizer
from nlp_common.acts_reader import ActsReader
from collections import Counter

### Data preparation

Data used in the exercise comes from two sources: set of polish bills (from previous exercies) and sgjp dictionary. The second one is loaded to elasticsearch in the attached script (main.py).

In [2]:
acts_reader = ActsReader('../ustawy') 
bills = [ text for _, _, text in acts_reader.all_acts()]

### Remove HTML tags

In [3]:
html_regex = regex.compile(r'<\/?[^>]*>', regex.IGNORECASE)
for bill in bills:
    matches = html_regex.findall(bill)
    for match in matches:
        print(match)

< < tajne >


Nothing to clean 🤷

### Tokenize

In [4]:
new_line_re = regex.compile(r'\n+')
bills = [new_line_re.sub(' ', bill) for bill in bills]

In [23]:
nlp = spacy.load("pl_core_news_sm")
prefix_re = regex.compile(r'''^[\[\("'\„]''')
suffix_re = regex.compile(r'''[\]\)"'\:\;\,\.]+$''')
nlp.tokenizer = Tokenizer(nlp.vocab, prefix_search=prefix_re.search, suffix_search=suffix_re.search)

In [24]:
bill_docs = list(nlp.tokenizer.pipe(bills))

### Word counters

In [25]:
ctrs = [ Counter([d.text.lower() for d in doc]) for doc in bill_docs ]

In [26]:
global_counter = Counter()
for ctr in ctrs:
    global_counter += ctr

In [27]:
global_counter

Counter({' ': 39644,
         'tekst': 329,
         'ustawy': 13067,
         'ustalony': 219,
         'ostatecznie': 103,
         'po': 13543,
         'rozpatrzeniu': 153,
         'poprawek': 222,
         'senatu': 346,
         '\xa0 \xa0 \xa0 ': 156,
         'ustawa': 3235,
         'z': 82419,
         'dnia': 17951,
         '11': 3531,
         'lipca': 1339,
         '2014': 251,
         'r': 33015,
         '.': 406743,
         '\xa0 ': 3834,
         'o': 64754,
         'zmianie': 1518,
         '–': 4002,
         'prawo': 4013,
         'ochrony': 4605,
         'środowiska': 3289,
         'oraz': 33556,
         'niektórych': 974,
         'innych': 4592,
         'ustaw[1]),[2': 1,
         '])': 836,
         '\xa0 \xa0 ': 235,
         'art': 83586,
         '\xa0': 10723,
         '1': 69281,
         'w': 201154,
         'ustawie': 5468,
         '27': 1925,
         'kwietnia': 993,
         '2001': 1993,
         '(': 12941,
         'dz': 2891,
         

### Rank

In [35]:
# Reject all entries that are shorter than 2 characters or contain non-letter characters (make sure to include Polish diacritics).
def reject(word):
    if len(word) < 2 or not regex.match(r'^\p{L}+$', word):
        return True
    return False

word_ctr_pairs = [ (word, global_counter[word]) for word in global_counter ]
word_ctr_pairs_without_rejected = filter(lambda w: not reject(w[0]), word_ctr_pairs)
print(list(word_ctr_pairs_without_rejected))

[('tekst', 329), ('ustawy', 13067), ('ustalony', 219), ('ostatecznie', 103), ('po', 13543), ('rozpatrzeniu', 153), ('poprawek', 222), ('senatu', 346), ('ustawa', 3235), ('dnia', 17951), ('lipca', 1339), ('zmianie', 1518), ('prawo', 4013), ('ochrony', 4605), ('środowiska', 3289), ('oraz', 33556), ('niektórych', 974), ('innych', 4592), ('art', 83586), ('ustawie', 5468), ('kwietnia', 993), ('dz', 2891), ('poz', 45009), ('wprowadza', 1999), ('się', 45864), ('następujące', 2276), ('zmiany', 4011), ('odnośnik', 6), ('nr', 44939), ('do', 60724), ('tytułu', 2914), ('otrzymuje', 9834), ('brzmienie', 10575), ('niniejsza', 67), ('dokonuje', 1153), ('zakresie', 6092), ('swojej', 260), ('regulacji', 277), ('wdrożenia', 117), ('następujących', 244), ('dyrektyw', 11), ('unii', 1209), ('europejskiej', 1462), ('dyrektywy', 194), ('rady', 4037), ('marca', 783), ('sprawie', 2193), ('ograniczania', 51), ('zanieczyszczenia', 188), ('azbestem', 2), ('zapobiegania', 213), ('temu', 191), ('zanieczyszczeniu', 